In [12]:
import pandas as pd
from flask import Flask, request, jsonify
from sklearn.metrics.pairwise import cosine_similarity
from flask_cors import CORS

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS

# Load the activity data
try:
    activity_data = pd.read_csv('Events.csv')  # Replace with your file name
    activity_data.columns = activity_data.columns.str.strip()  # Clean column names
    activity_data['Participants'] = activity_data['Participants'].astype(int)  # Ensure numeric type for Participants
except Exception as e:
    raise RuntimeError(f"Failed to load activity data: {e}")

# One-hot encode features for cosine similarity
activity_features = pd.get_dummies(activity_data[['Duration', 'Category', 'Participants', 'SDG']])

# In-memory storage to track user sessions and suggested activities
session_store = {}

@app.route('/recommend', methods=['POST'])
def recommend():
    try:
        input_data = request.get_json()

        if not input_data:
            return jsonify({"error": "Input data is required"}), 400

        print(input_data)
        # Ensure at least 2 features are provided
        required_features = ['Duration', 'Category', 'Participants', 'SDG']
        selected_features = [key for key in required_features if key in input_data and input_data[key]]
        
        if len(selected_features) < 2:
            return jsonify({"error": "Please select at least 2 features for filtering."}), 400
        
        # Create a unique session ID based on input data (you can use client-specific identifiers)
        session_id = str(input_data)
        
        # Filter the dataset based on selected features
        filtered_data = activity_data.copy()
        strict_match_count = 0
        
        for feature in selected_features:
            filtered_data = filtered_data[filtered_data[feature] == input_data[feature]]
            strict_match_count += 1

            # Stop filtering after two strict matches
            if strict_match_count == 2:
                break
        for feature in selected_features:
            filtered_data = filtered_data[filtered_data[feature] == input_data[feature]]
        
        # If no matching activities are found, use the entire dataset as a fallback
        if filtered_data.empty:
            filtered_data = activity_data
        
        # One-hot encode input data for similarity scoring
        input_df = pd.DataFrame([input_data])
        input_features = pd.get_dummies(input_df)
        input_features = input_features.reindex(columns=activity_features.columns, fill_value=0)

        # Calculate cosine similarity with the filtered (or entire) dataset
        filtered_features = pd.get_dummies(filtered_data[['Duration', 'Category', 'Participants', 'SDG']])
        filtered_features = filtered_features.reindex(columns=activity_features.columns, fill_value=0)

        similarity_scores = cosine_similarity(input_features, filtered_features)

        # Rank activities by weighted similarity
        weights = {'Category': 4, 'Duration': 2, 'Participants': 2, 'SDG': 1}
        weighted_scores = (
            similarity_scores[0] *
            filtered_features.mul(
                filtered_features.columns.map(
                    lambda col: weights.get(col.split('_')[0], 1)
                ),
                axis=1
            ).sum(axis=1).values
        )
        
        # Track already suggested indices for this session
        if session_id not in session_store:
            session_store[session_id] = set()  # Initialize session storage

        # Exclude previously suggested activities
        available_indices = [i for i in range(len(filtered_data)) if i not in session_store[session_id]]

        # If no more activities are available, reset the session for cyclic suggestions
        if not available_indices:
            session_store[session_id].clear()  # Reset the session store
            available_indices = list(range(len(filtered_data)))

        # Get top 5 activities based on similarity
        top_indices = sorted(available_indices, key=lambda i: weighted_scores[i], reverse=True)[:5]
        matched_activities = filtered_data.iloc[top_indices].to_dict(orient='records')

        # If fewer than 5 activities are matched, fill the remainder with fallback activities
        if len(matched_activities) < 5:
            fallback_data = activity_data[~activity_data.index.isin(filtered_data.index)]
            fallback_features = pd.get_dummies(fallback_data[['Duration', 'Category', 'Participants', 'SDG']])
            fallback_features = fallback_features.reindex(columns=activity_features.columns, fill_value=0)
            fallback_similarity_scores = cosine_similarity(input_features, fallback_features)
            fallback_top_indices = fallback_similarity_scores[0].argsort()[::-1][: (5 - len(matched_activities))]
            fallback_activities = fallback_data.iloc[fallback_top_indices].to_dict(orient='records')
            matched_activities.extend(fallback_activities)

        # Update session store with the indices of suggested activities
        session_store[session_id].update(top_indices)
        print(matched_activities)

        return jsonify(matched_activities)
    except Exception as e:
        return jsonify({"error": f"An error occurred: {str(e)}"}), 500


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.100.96:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Jan/2025 01:08:36] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2025 01:08:36] "POST /recommend HTTP/1.1" 200 -


{'Category': '', 'Duration': '2-3 hours', 'Participants': '', 'SDG': 'Industry, Innovation, and Infrastructure'}
[{'Activity': 'Yoga classes', 'Duration': '2-3 hours', 'Category': 'Arts and Creativity', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Research project presentations', 'Duration': '2-3 hours', 'Category': 'Community Engagement', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Dorm badminton league', 'Duration': '2-3 hours', 'Category': 'Arts and Creativity', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Social media branding workshops', 'Duration': '2-3 hours', 'Category': 'Innovation and Technology', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Fan art contests', 'Duration': '2-3 hours', 'Category': 'Cultural Exchange', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}]


127.0.0.1 - - [04/Jan/2025 01:08:37] "POST /recommend HTTP/1.1" 200 -


{'Category': '', 'Duration': '2-3 hours', 'Participants': '', 'SDG': 'Industry, Innovation, and Infrastructure'}
[{'Activity': 'Archery competitions', 'Duration': '2-3 hours', 'Category': 'Arts and Creativity', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'App-based dorm management ideas', 'Duration': '2-3 hours', 'Category': 'Sports and Recreation', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Drama or skit night', 'Duration': '2-3 hours', 'Category': 'Sports and Recreation', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Dorm storytelling evenings', 'Duration': '2-3 hours', 'Category': 'Arts and Creativity', 'Participants': 20, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Biking expeditions', 'Duration': '2-3 hours', 'Category': 'Community Engagement', 'Participants': 20, 'SDG': 'Industry, Innovation, and Infrastructure'}]


...

{'Category': 'Cultural Exchange', 'Duration': 'Half day', 'Participants': '', 'SDG': ''}
[{'Activity': 'DIY dorm café day', 'Duration': 'Half day', 'Category': 'Cultural Exchange', 'Participants': 150, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Reusable bag-making sessions', 'Duration': '1 week', 'Category': 'Cultural Exchange', 'Participants': 15, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Dorm custom sticker printing', 'Duration': 'Full day', 'Category': 'Cultural Exchange', 'Participants': 15, 'SDG': 'Reduced Inequalities'}, {'Activity': 'Handmade soap sessions', 'Duration': 'Half day', 'Category': 'Health and Well-being', 'Participants': 15, 'SDG': 'Climate Action'}, {'Activity': 'Sand volleyball', 'Duration': 'Half day', 'Category': 'Community Engagement', 'Participants': 15, 'SDG': 'Climate Action'}]


127.0.0.1 - - [04/Jan/2025 01:33:04] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2025 01:33:04] "POST /recommend HTTP/1.1" 200 -


{'Category': 'Community Engagement', 'Duration': 'Full day', 'Participants': '', 'SDG': ''}
[{'Activity': 'Meditation under the stars', 'Duration': 'Full day', 'Category': 'Community Engagement', 'Participants': 15, 'SDG': 'Partnerships for the Goals'}, {'Activity': 'Dorm time capsule', 'Duration': 'Full day', 'Category': 'Community Engagement', 'Participants': 15, 'SDG': 'Climate Action'}, {'Activity': 'Calligraphy workshops', 'Duration': 'Full day', 'Category': 'Community Engagement', 'Participants': 20, 'SDG': 'Industry, Innovation, and Infrastructure'}, {'Activity': 'Video blogging sessions', 'Duration': 'Full day', 'Category': 'Community Engagement', 'Participants': 80, 'SDG': 'Reduced Inequalities'}, {'Activity': 'Sticker design challenges', 'Duration': 'Full day', 'Category': 'Community Engagement', 'Participants': 80, 'SDG': 'Reduced Inequalities'}]


127.0.0.1 - - [04/Jan/2025 01:35:46] "OPTIONS /recommend HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2025 01:35:46] "POST /recommend HTTP/1.1" 200 -


{'Category': 'Cultural Exchange', 'Duration': 'Full day', 'Participants': '', 'SDG': ''}
[{'Activity': 'Dorm custom sticker printing', 'Duration': 'Full day', 'Category': 'Cultural Exchange', 'Participants': 15, 'SDG': 'Reduced Inequalities'}, {'Activity': 'DIY slime-making workshop', 'Duration': 'Full day', 'Category': 'Cultural Exchange', 'Participants': 20, 'SDG': 'Climate Action'}, {'Activity': 'Astronomy stargazing sessions', 'Duration': 'Full day', 'Category': 'Cultural Exchange', 'Participants': 20, 'SDG': 'Climate Action'}, {'Activity': 'Dorm pasta night', 'Duration': 'Full day', 'Category': 'Cultural Exchange', 'Participants': 20, 'SDG': 'Reduced Inequalities'}, {'Activity': 'Dorm stretching breaks', 'Duration': 'Full day', 'Category': 'Cultural Exchange', 'Participants': 40, 'SDG': 'Good Health and Well-being'}]
